In [ ]:
# We have been provided with a dataset which consist of credit card details their transaction and 
#complete behavioural pattern of a user.Our task is group them into clusters based on 
#their activites in the past. 


In [ ]:
#import all the required libraries 
import pandas as pd 
import numpy as np
import seaborn as sns
from sklearn.preprocessing import PowerTransformer,StandardScaler
from sklearn.decomposition import PCA
from scipy.stats import zscore

In [ ]:
df = pd.read_csv('../input/ccdata/CC GENERAL.csv')


In [ ]:
df.head()

In [ ]:
df.describe().T

In [ ]:
df.info()

In [ ]:
df.isnull().sum()/df.shape[0]

In [ ]:
#Fill those null values with mean 

In [ ]:
df['MINIMUM_PAYMENTS']=df['MINIMUM_PAYMENTS'].fillna(value=df['MINIMUM_PAYMENTS'].mean())
df['CREDIT_LIMIT']=df['CREDIT_LIMIT'].fillna(value=df['CREDIT_LIMIT'].mean())

In [ ]:
#check for skewness

In [ ]:
import matplotlib.pyplot as plt 

In [ ]:
for i in df.select_dtypes(['int','float64']).columns:
    sns.distplot(df[i])
    plt.show()

In [ ]:
#sing power transformer to deal with the outliers.

In [ ]:
x=df.drop('CUST_ID',axis=1,inplace=True)

In [ ]:
from sklearn.preprocessing import PowerTransformer

pt = PowerTransformer()

x_kmean = pt.fit_transform(df)
x_kmean = pd.DataFrame(x_kmean,columns=df.columns)

In [ ]:
for i in x_kmean.select_dtypes(['int','float64']).columns:
    sns.distplot(x_kmean[i])
    plt.show()

In [ ]:
x_kmean

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=[12,12])
sns.heatmap(df.corr(), annot=True)


# As we find that data  has mulitcollinearity we can deal with this using PCA. it Uses the information content that is delveloped by the two independant variables that is developed coherently.we take the independant variables and standardize the data.we then capture the information between the independant variables x1 and x2 using covariance matrix.(Note-This information would not have been captured if we had passed x as just an independant variable).Now we apply eigen funtion or decomposition,We get eigen vectors and eigen values. Eigen vector is nothing but the new axes. (The new axes is the orthogonal axes after standardazing and capturing the covariance matrix) We get 2 eigen vectors for x1 and x2 which are the new components.The variance captured by one vector is eigen value.Here we are passing the 95% to keep the number of components which captures 95%variance.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(x_kmean)


Now we are going to build our machine learning clustering models. 

we will be using two methods; **k-means clustering** and **hierarchical clustering/Agglomerative clustering** algorithms.

For k-means clustering algorithm:

* First of all I will find the best k value.
* Than I will use this k value to create a k-means model.
* And I will compare my original and k-means clustered datas.

For hierarchical clustering algorithm:

* First I will apply a dendrogram in order to find how many classes do I have in my data.
* Than I will use this class number to apply a hierarchical clustering algorithm.
* Lastly I will compare my original, k-means and hierarchical clustered datas.


# K means clustering

In [ ]:
from sklearn.cluster import KMeans

cluster_range = np.arange(1,16)
cluster_error = []

for i in cluster_range:
    k = KMeans(n_clusters=i,n_init=14)
    k.fit(x_kmean)
    cluster_error.append(k.inertia_)
    
k_means = [KMeans(n_clusters=i,random_state=4).fit(x_kmean) for i in range(1,16)]

In [ ]:
pd.DataFrame({'No of Clusters':cluster_range,'Cluster Errors':cluster_error})

In [ ]:
plt.plot(cluster_range,cluster_error,marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Cluster Error')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=23)
kmeans.fit(X_reduced)


In [ ]:
df['label']=kmeans.labels_

In [ ]:
plt.title('Predicted Classes')
sns.scatterplot(data=df,x='CREDIT_LIMIT', y='ONEOFF_PURCHASES', hue='label')
plt.show()

In [ ]:
plt.title('Predicted Classes')
sns.scatterplot(data=df,x='PAYMENTS', y='BALANCE_FREQUENCY', hue='label')
plt.show() 

In [ ]:
plt.title('Predicted Classes')
sns.scatterplot(data=df,x='PAYMENTS', y='PURCHASES_TRX', hue='label')
plt.show() 

In [ ]:
plt.title('Predicted Classes')
sns.scatterplot(data=df,x='PAYMENTS', y='ONEOFF_PURCHASES', hue='label')
plt.show()

# Agglomerative clustering

In [ ]:
from sklearn.preprocessing import PowerTransformer

In [ ]:
df.drop('CUST_ID',axis=1,inplace=True)

In [ ]:
pt = PowerTransformer()

x_ag = pt.fit_transform(df)
x_ag = pd.DataFrame(x_ag,columns=df.columns)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
X_reducedagg = pca.fit_transform(x_ag)


In [ ]:
from scipy.cluster.hierarchy import dendrogram,linkage
plt.figure(figsize=(25,30))
merg=linkage(X_reducedagg,method='ward')
dendrogram(merg,p=30,leaf_rotation=90,color_threshold=25,leaf_font_size=10,truncate_mode='level')
plt.show()

In [ ]:
from sklearn.cluster import AgglomerativeClustering
hie_clus=AgglomerativeClustering(n_clusters=3,affinity='euclidean',linkage='ward')


In [ ]:
hie_clus.fit(X_reducedagg)

In [ ]:
df['class'] = hie_clus.labels_

In [ ]:
#Please note that lables is for k means and class denotes agglomerative clustering 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.title('Predicted Classes')
sns.scatterplot(data=df,x='PAYMENTS', y='ONEOFF_PURCHASES', hue='class')
plt.show()

In [ ]:
plt.title('Predicted Classes')
sns.scatterplot(data=df,x='CREDIT_LIMIT', y='ONEOFF_PURCHASES', hue='class')
plt.show()

In [ ]:
#agglomerative clustering  
df.groupby('class').size()

In [ ]:
#kmeans clustering
df.groupby('label').size()